In [1]:
!pip install --upgrade simpy 

Taller 9: Usando la implementación en SimPy del ejemplo de Bank Renege en Notebook Jupyter, implementar:
1) La medida de desempeño de tiempo promedio en fila de los clientes (Xi) para una corrida de simulación i .
2) Implementar el método recursivo para calcular la media de la muestra y la varianza de la muestra.
3) Calcular la media de la muestra (sample mean) y la varianza de la muestra (sample variance) para:
k=100 simulaciones.
k=1000 simulaciones.
k=10000 simulaciones.

In [12]:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

N_SIMU = 100 
media = []
varianza = []
times = [] 
Xn = []
aux = 0
x_i = 0
s_i = 0

def Sample_variance(lista, X):
    S = 0
    for i in range(len(lista)):
        S = S + (lista[i] - X)**2
        
    return S/(len(lista) - 1)

def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    global times
    """Customer arrives, is served and leaves."""
    arrive = env.now
    #print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        times.append(wait)
        
        if req in results:
            # We got to the counter
            
            #print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            #print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            #print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            times.append(0)
while(True):
    for i in range(N_SIMU):
        times.clear()
        print("Muestra No. ", i+1)
        print("Clientes: ", NEW_CUSTOMERS)

        # Setup and start the simulation
        env = simpy.Environment()

        # Start processes and run
        counter = simpy.Resource(env, capacity=1)
        env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
        env.run()
        #print("-------------------------------")
        xn = sum(times)/NEW_CUSTOMERS
        Xn.append(xn)
        print("1. Tiempo promedio de los clientes en fila -> ", xn)
        aux = x_i
        x_i = x_i + (xn - x_i)/(i+1)
        print("2.1. Promedio de la muestra -> ", x_i)

        if i>=1:
            s_i = (1 - 1/i)*s_i + (i + 1)*(x_i - aux)**2
            print("2.2 Varianza de la muestra -> ", s_i)
            print("")

        
    
    #Sample_mean and Sample_variance
    if N_SIMU == 100 or N_SIMU == 1000 or N_SIMU == 10000:
        X = sum(Xn)/len(Xn)
        media.append(X)
        varianza.append(Sample_variance(Xn, X))
    
    N_SIMU = N_SIMU * 10
    Xn = []
    if N_SIMU > 10000:
        break
        
print ("3.")
for u in range(len(media)):
    print ("Media de la muestra para k = ",100*(10**u), " -> ", media[u])
    print ("Varianza de la muestra para k = ",100*(10**u), " -> ", varianza[u])
    print ("")



Muestra No.  1
Clientes:  5  clientes
1. Tiempo promedio de los clientes en fila ->  0.31439859440757995
2.1. Promedio de la muestra ->  0.31439859440757995
Muestra No.  2
Clientes:  5  clientes
1. Tiempo promedio de los clientes en fila ->  0.8530352649901676
2.1. Promedio de la muestra ->  0.5837169296988738
2.2 Varianza de la muestra ->  0.14506473144814755

Muestra No.  3
Clientes:  5  clientes
1. Tiempo promedio de los clientes en fila ->  0.5627254316367057
2.1. Promedio de la muestra ->  0.5767197636781511
2.2 Varianza de la muestra ->  0.07267924672103844

Muestra No.  4
Clientes:  5  clientes
1. Tiempo promedio de los clientes en fila ->  0.4976702482044232
2.1. Promedio de la muestra ->  0.5569573848097191
2.2 Varianza de la muestra ->  0.05001503762151676

Muestra No.  5
Clientes:  5  clientes
1. Tiempo promedio de los clientes en fila ->  1.227601977345413
2.1. Promedio de la muestra ->  0.6910863033168579
2.2 Varianza de la muestra ->  0.127464112115611

Muestra No.  6
Cli